In [1]:
import os
import glob
import pandas as pd
import numpy as np

from transformers4rec import tf as tr
import tensorflow as tf
from transformers4rec.tf.ranking_metric import NDCGAt, RecallAt

In [2]:
INPUT_DATA_DIR = os.environ.get("INPUT_DATA_DIR", '../data/')

In [5]:
from merlin_standard_lib import Schema
# define schema object to pass it to the TabularSeqeunceFeatures class
SCHEMA_PATH = os.path.join(INPUT_DATA_DIR, 'schema_test.pb')
schema = Schema().from_proto_text(SCHEMA_PATH)

In [6]:
sequence_length = 20
inputs = tr.TabularSequenceFeatures.from_schema(
schema,
max_sequence_length = sequence_length,
aggregation = 'concat',
)

In [7]:
def tf_yoochoose_like():
    return tr.data.tabular_sequence_testing_data.tf_synthetic_data(
    num_rows=100, min_session_length=5, max_session_length=20
    )

In [8]:
data = tf_yoochoose_like()
body = tr.SequentialBlock([inputs, tr.MLPBlock([64])])
targets = {"target": tf.cast(tf.random.uniform((100,), maxval=2, dtype=tf.int32), tf.float32)}
model = tr.BinaryClassificationTask("target").to_model(body, inputs)
model.compile(optimizer="adam", run_eagerly=False)
dataset = tf.data.Dataset.from_tensor_slices((data, targets)).batch(50)
losses = model.fit(dataset, epochs=5)

2021-12-03 13:04:08.529344: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 1/5
2/2 [==============================] - 3s 22ms/step - target/binary_classification_task/precision: 0.4889 - target/binary_classification_task/recall: 0.4400 - target/binary_classification_task/binary_accuracy: 0.4900 - target/binary_classification_task/auc: 0.4854 - loss: 0.6944 - regularization_loss: 0.0000e+00 - total_loss: 0.6944
Epoch 2/5
2/2 [==============================] - 0s 22ms/step - target/binary_classification_task/precision: 0.5714 - target/binary_classification_task/recall: 0.3200 - target/binary_classification_task/binary_accuracy: 0.5400 - target/binary_classification_task/auc: 0.6220 - loss: 0.6848 - regularization_loss: 0.0000e+00 - total_loss: 0.6848
Epoch 3/5
2/2 [==============================] - 0s 20ms/step - target/binary_classification_task/precision: 0.7308 - target/binary_classification_task/recall: 0.3800 - target/binary_classification_task/binary_accuracy: 0.6200 - target/binary_classification_task/auc: 0.7028 - loss: 0.6771 - regularization_los